In [17]:
import triton 
import torch
import triton.language as tl

In [21]:
@triton.jit 
def maxminelement(inptr,maxans,minans,m,blocksize:tl.constexpr):
    blockid=tl.program_id(0)
    block_start=blockid*blocksize
    offsets = block_start + tl.arange(0, blocksize)
    mask = offsets < m
    vals = tl.load(inptr + offsets, mask=mask)
    block_max = tl.max(vals, axis=0)
    block_min = tl.min(vals, axis=0)
    if blockid == 0:
        tl.store(maxans + 0, block_max)  
        tl.store(minans + 0, block_min)  


In [22]:
def test():
   M=10
   a = torch.randn(M, device='cuda', dtype=torch.float16)
   maxans = torch.empty(1, device='cuda')
   minans = torch.empty(1, device='cuda')
   blocksize = 128
   noofblocks = triton.cdiv(M, blocksize)  
   
   maxminelement[(noofblocks,)](a, maxans,minans, M,blocksize)  
   
   print(maxans)
   print(minans)

In [23]:
if __name__=='__main__':
    test()

tensor([1.6328], device='cuda:0')
tensor([-2.6777], device='cuda:0')
